In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import pinecone




c:\Users\sjufa\anaconda3\envs\py39\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Import environment variables
import os   
from dotenv import load_dotenv
load_dotenv()

# Establish environment variables
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_KEY")
os.environ["PINECONE_ENV"] = os.getenv("PINECONE_ENV")
os.environ["OPENAI_ORGANIZATION"] = os.getenv("OPENAI_ORG")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")





In [4]:
# Load in additional training resources
additional_loader = TextLoader("menu_tests.txt")
additional_documents = additional_loader.load()

# Load in user-uploaded menu
menu_loader = TextLoader("menu_tests.txt")
menu_documents = menu_loader.load()

# Combine documents
documents = additional_documents + menu_documents

# Initialize pinecone
pinecone.init(
    api_key=os.getenv("PINECONE_KEY"), 
    environment=os.getenv("PINECONE_ENV")
    )




In [7]:
# Split and embed the documents
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

index_name = "training-resources"

vectorstore = Pinecone.from_documents(documents, embeddings, index_name = index_name)



In [18]:
# Initialize the ConversationalRetrievalChain
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(model_name = "gpt-3.5-turbo",temperature=0), vectorstore.as_retriever(), memory=memory)

In [ ]:
def get_chat_history(inputs) -> str:
    res = []
    for human, ai in inputs:
        res.append(f"Human:{human}\nAI:{ai}")
    return "\n".join(res)
qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0), vectorstore.as_retriever(), get_chat_history=get_chat_history)

In [20]:
# Ask a question
query = "If I gave you the answers to those questions could you grade them for me?"
result = qa({"question": query})
print(result["answer"])

Yes, I can grade a five-question test based on the information in the training resources if you provide me with the answers.


In [21]:
messages = memory.chat_memory.messages
print(messages)

[HumanMessage(content='Can you write a quick five question test based on the information in the training resources?', additional_kwargs={}, example=False), AIMessage(content="Sure, here's a quick five-question test based on the information in the training resources:\n\n1. What are some techniques used in bartending?\n2. How should you handle a difficult customer?\n3. What are the two primary ways wines are classified?\n4. Why is food safety important in a restaurant or bar?\n5. When should cocktails containing clear spirits and no mixers be stirred instead of shaken?", additional_kwargs={}, example=False), HumanMessage(content='If I gave you the answers to those questions could you grade them for me?', additional_kwargs={}, example=False), AIMessage(content='Yes, I can grade a five-question test based on the information in the training resources if you provide me with the answers.', additional_kwargs={}, example=False)]


In [26]:
messages[0].content


'Can you write a quick five question test based on the information in the training resources?'

In [38]:
# If the message in the chat history is an "AIMessage", print the AI message
print(messages)


[HumanMessage(content='Can you write a quick five question test based on the information in the training resources?', additional_kwargs={}, example=False), AIMessage(content="Sure, here's a quick five-question test based on the information in the training resources:\n\n1. What are some techniques used in bartending?\n2. How should you handle a difficult customer?\n3. What are the two primary ways wines are classified?\n4. Why is food safety important in a restaurant or bar?\n5. When should cocktails containing clear spirits and no mixers be stirred instead of shaken?", additional_kwargs={}, example=False), HumanMessage(content='If I gave you the answers to those questions could you grade them for me?', additional_kwargs={}, example=False), AIMessage(content='Yes, I can grade a five-question test based on the information in the training resources if you provide me with the answers.', additional_kwargs={}, example=False)]


In [43]:
for message in messages:
    if message.type == 'human':
        print(f"Human: {message.content}")
    else:
        print(f"AI: {message.content}")

Human: Can you write a quick five question test based on the information in the training resources?
AI: Sure, here's a quick five-question test based on the information in the training resources:

1. What are some techniques used in bartending?
2. How should you handle a difficult customer?
3. What are the two primary ways wines are classified?
4. Why is food safety important in a restaurant or bar?
5. When should cocktails containing clear spirits and no mixers be stirred instead of shaken?
Human: If I gave you the answers to those questions could you grade them for me?
AI: Yes, I can grade a five-question test based on the information in the training resources if you provide me with the answers.


In [47]:
import pandas as pd
import json

file_path = "recipes.json"

with open(file_path, 'r') as f:
    data = f.readlines()

data

'''# Iterate over each line in the file and load as a DataFrame
dfs = []
for line in data:
    json_data = json.loads(line)
    df = pd.DataFrame(json_data)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dfs)

print(combined_df)'''


'# Iterate over each line in the file and load as a DataFrame\ndfs = []\nfor line in data:\n    json_data = json.loads(line)\n    df = pd.DataFrame(json_data)\n    dfs.append(df)\n\n# Concatenate all DataFrames into a single DataFrame\ncombined_df = pd.concat(dfs)\n\nprint(combined_df)'

In [49]:
type(data)

list

In [55]:
# For each string in the list, convert to a dictionary
data_dicts = []
for line in data:
    data_dict = json.loads(line)
    data_dicts.append(data_dict)

data_dicts

[{'Name': 'Christmas pie',
  'url': 'https://www.bbcgoodfood.com/recipes/2793/christmas-pie',
  'Description': 'Combine a few key Christmas flavours here to make a pie that both children and adults will adore',
  'Author': 'Mary Cadogan',
  'Ingredients': ['2 tbsp olive oil',
   'knob butter',
   '1 onion, finely chopped',
   '500g sausagemeat or skinned sausages',
   'grated zest of 1 lemon',
   '100g fresh white breadcrumbs',
   '85g ready-to-eat dried apricots, chopped',
   '50g chestnut, canned or vacuum-packed, chopped',
   '2 tsp chopped fresh or 1tsp dried thyme',
   '100g cranberries, fresh or frozen',
   '500g boneless, skinless chicken breasts',
   '500g pack ready-made shortcrust pastry',
   'beaten egg, to glaze'],
  'Method': ['Heat oven to 190C/fan 170C/gas 5. Heat 1 tbsp oil and the butter in a frying pan, then add the onion and fry for 5 mins until softened. Cool slightly. Tip the sausagemeat, lemon zest, breadcrumbs, apricots, chestnuts and thyme into a bowl. Add the o

In [56]:
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data_dicts)
df

,Name,url,Description,Author,Ingredients,Method
0,Christmas pie,https://www.bbcgoodfood.com/recipes/2793/chris...,Combine a few key Christmas flavours here to m...,Mary Cadogan,"[2 tbsp olive oil, knob butter, 1 onion, finel...",[Heat oven to 190C/fan 170C/gas 5. Heat 1 tbsp...
1,Simmer-&-stir Christmas cake,https://www.bbcgoodfood.com/recipes/1160/simme...,An easy-to-make alternative to traditional Chr...,Mary Cadogan,"[175g butter, chopped, 200g dark muscovado sug...","[Put the butter, sugar, fruit, zests, juice an..."
2,Christmas cupcakes,https://www.bbcgoodfood.com/recipes/72622/chri...,These beautiful and classy little cakes make l...,Sara Buenfeld,"[200g dark muscovado sugar, 175g butter, chopp...","[Tip the sugar, butter, dried fruit, whole che..."
3,Christmas buns,https://www.bbcgoodfood.com/recipes/1803633/ch...,Paul Hollywood's fruit rolls can be made ahead...,Paul Hollywood,"[500g strong white flour, plus extra for dusti...",[Put the flour and 1 tsp salt into a large bow...
4,Christmas cupcakes,https://www.bbcgoodfood.com/recipes/981634/chr...,"Made these for the second time today, and I ha...",Barney Desmazery,"[280g self-raising flour, 175g golden caster s...",[Heat oven to 190C/170 fan/gas 5 and line a 12...
...,...,...,...,...,...,...
1612,Flavoured butters,https://www.bbcgoodfood.com/recipes/1911/flavo...,Try these different flavoured butters melted o...,Maxine Clark,"[100g softened butter, 2-3 tbsp chopped fresh ...",[Take100g/4oz softened butter and beat until c...
1613,Crispy Roast Duck,https://www.bbcgoodfood.com/recipes/3248/crisp...,"Duck basted to perfection, ideal for a Sunday ...",Ruth Watson,"[3 medium-sized duckss (each about 2¼kg/5lb), ...",[Remove the duck’s giblets and wash them. Cut ...
1614,Smoked salmon kedgeree,https://www.bbcgoodfood.com/recipes/3047/smoke...,This kedgeree is totally delicious. Treat ever...,Barney Desmazery,"[300g basmati rice, 8 eggs, 400g pack sliced s...","[To make the sauce, heat the butter in a large..."
1615,Spicy potted crab,https://www.bbcgoodfood.com/recipes/1797/spicy...,An ideal elegant starter for a wedding,Merrilees Parker,"[6 x 250g pkts unsalted butter, 2 tsp hot chil...",[ Heat the butter in a pan until melted. Turn ...


In [57]:
from langchain.agents import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"), df, verbose=True)
agent.run("What are two recipes that contain salmon?")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)).




> Entering new AgentExecutor chain...
Thought: We need to search for recipes that contain salmon in the dataframe.
Action: python_repl_ast
Action Input: df[df['Ingredients'].str.contains('salmon')]
Observation: "None of [Float64Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n              ...\n              nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n             dtype='float64', length=1617)] are in the [columns]"
Thought:It seems like the 'Ingredients' column may not have been properly formatted or there are no recipes in the dataframe that contain salmon.
Action: python_repl_ast
Action Input: df.columns
Observation: Index(['Name', 'url', 'Description', 'Author', 'Ingredients', 'Method'], dtype='object')
Thought:The 'Ingredients' column is present in the dataframe, so it must be an issue with the formatting of the data in that column.
Action: python_repl_ast
Action Input: df[df['Ingredients'].notnull() & df['Ingredients'].str.contains('salmon')]
Observation: Empty

'There are no recipes in the dataframe that contain salmon.'

In [62]:
df[df['Ingredients'].notnull()].head(10)

,Name,url,Description,Author,Ingredients,Method
0,Christmas pie,https://www.bbcgoodfood.com/recipes/2793/chris...,Combine a few key Christmas flavours here to m...,Mary Cadogan,"[2 tbsp olive oil, knob butter, 1 onion, finel...",[Heat oven to 190C/fan 170C/gas 5. Heat 1 tbsp...
1,Simmer-&-stir Christmas cake,https://www.bbcgoodfood.com/recipes/1160/simme...,An easy-to-make alternative to traditional Chr...,Mary Cadogan,"[175g butter, chopped, 200g dark muscovado sug...","[Put the butter, sugar, fruit, zests, juice an..."
2,Christmas cupcakes,https://www.bbcgoodfood.com/recipes/72622/chri...,These beautiful and classy little cakes make l...,Sara Buenfeld,"[200g dark muscovado sugar, 175g butter, chopp...","[Tip the sugar, butter, dried fruit, whole che..."
3,Christmas buns,https://www.bbcgoodfood.com/recipes/1803633/ch...,Paul Hollywood's fruit rolls can be made ahead...,Paul Hollywood,"[500g strong white flour, plus extra for dusti...",[Put the flour and 1 tsp salt into a large bow...
4,Christmas cupcakes,https://www.bbcgoodfood.com/recipes/981634/chr...,"Made these for the second time today, and I ha...",Barney Desmazery,"[280g self-raising flour, 175g golden caster s...",[Heat oven to 190C/170 fan/gas 5 and line a 12...
5,Christmas slaw,https://www.bbcgoodfood.com/recipes/890635/chr...,"A nutty winter salad which is superhealthy, qu...",Good Food,"[2 carrots, halved, ½ white cabbage, shredded,...",[Peel strips from the carrots using a vegetabl...
6,Christmas mess,https://www.bbcgoodfood.com/recipes/2806664/ch...,Delicious and a synch to make! Have made this ...,Caroline Hire,"[600ml double cream, 400g Greek yoghurt, 4 tbs...",[In a small saucepan gently heat the frozen be...
7,Christmas brownies,https://www.bbcgoodfood.com/recipes/christmas-...,can I made these the day before,Miriam Nice,"[200g unsalted buttercut into cubes, plus extr...",[Grease and line a 20cm x 20cm brownie tin. He...
8,Christmas cosmopolitan,https://www.bbcgoodfood.com/recipes/889643/chr...,How many servings have you gotten using the ab...,Good Food,"[500ml vodka, 500ml gingerwine, 1l cranberry j...",[Mix the vodka and ginger wine in a jug. Stir ...
9,Christmas pizza,https://www.bbcgoodfood.com/recipes/christmas-...,Use up leftover roast turkey and sausagemeat s...,Katy Greenwood,"[145g pizza basemix, 6 tbsp tomato pasta sauce...",[Heat oven to 220C/200C fan/gas 7. Prepare the...
